<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/13_1_compilers_and_interpreters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.7 MB/s eta 0:00:00


## 13.1 Compilers and Interpreters

In [2]:
def add(a, b):
  return a + b

def fancy_func(a, b, c, d):
  e = add(a, b)
  f = add(c, d)
  g = add(e, f)
  return g

In [3]:
print(fancy_func(1, 2, 3, 4))

10


### 13.1.1 Symbolic Programming

In [5]:
def add_():
  return  '''
def add(a, b):
  return a + b
'''

def fancy_func_():
  return '''
def fancy_func(a, b, c, d):
  e = add(a, b)
  f = add(c, d)
  g = add(e, f)
  return g
'''

def evoke_():
  return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)



def add(a, b):
  return a + b

def fancy_func(a, b, c, d):
  e = add(a, b)
  f = add(c, d)
  g = add(e, f)
  return g
print(fancy_func(1, 2, 3, 4))
10


### 13.1.3 Hybridizing the `sequential` Class

In [6]:
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
# Factory for networks
def get_net():
  net = nn.Sequential(nn.Linear(512, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Linear(128, 2))
  return net

In [8]:
x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.0716,  0.0223]], grad_fn=<AddmmBackward0>)

In [9]:
net = torch.jit.script(net)
net(x)

tensor([[-0.0716,  0.0223]], grad_fn=<AddmmBackward0>)

#### Acceleration by Hybridization

In [10]:
class Benchmark:
  def __init__(self, description='Done'):
    self.description = description

  def __enter__(self):
    self.timer = d2l.Timer()
    return self

  def __exit__(self, *args):
    print(f'{self.description}: {self.timer.stop(): .4f} sec')

In [11]:
net = get_net()
with Benchmark('Without torchscript'):
  for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('With torchscript'):
  for i in range(1000): net(x)

Without torchscript:  0.1671 sec
With torchscript:  0.1250 sec


#### Serialization

In [14]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-r--r-- 1 root root 651K May  7 05:48 my_mlp
